# SSD MobileNet v2 Training on Custom TFRecord Dataset (Roboflow + Colab)
This notebook shows how to train an object detection model using TensorFlow's SSD MobileNet v2 on a labeled dataset in TFRecord format, fetched from Roboflow.

In [ ]:
# ✅ Install required libraries
!pip install roboflow --quiet
!pip install tensorflow_io --quiet
!pip install tf_slim --quiet
!pip install lxml --quiet
!pip install pycocotools --quiet

In [ ]:
# ✅ Import necessary libraries
from roboflow import Roboflow
import os
import shutil
import tensorflow as tf

In [ ]:
# ✅ Download dataset from Roboflow
# Replace with your Roboflow API key and project/version
rf = Roboflow(api_key="<YOUR_API_KEY>")
project = rf.workspace("<WORKSPACE_NAME>").project("<PROJECT_NAME>")
dataset = project.version(<VERSION_NUMBER>).download("tfrecord")

In [ ]:
# ✅ Clone TensorFlow models repository (if not already)
!git clone --depth 1 https://github.com/tensorflow/models
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python3 -m pip install . --quiet
%cd ../..

In [ ]:
# ✅ Download and extract pre-trained model checkpoint
!rm -rf pre-trained-model
!mkdir pre-trained-model
%cd pre-trained-model
!wget https://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xvzf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
%cd ..

In [ ]:
# ✅ Prepare pipeline config
# Copy config to training directory
!mkdir training
!cp pre-trained-model/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config training/ssd_mobilenet_v2.config

👉 Open `training/ssd_mobilenet_v2.config` and modify the following fields:
- `fine_tune_checkpoint`: Path to `.ckpt` in the pre-trained model folder
- `train_input_reader` > `label_map_path`: path to `label_map.pbtxt`
- `train_input_reader` > `input_path`: path to `train.record`
- `eval_input_reader` > `input_path`: path to `valid.record`
- `num_classes`: total number of classes in your dataset

In [ ]:
# ✅ Start training (adjust config and directory if needed)
!python models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path=training/ssd_mobilenet_v2.config \
  --model_dir=training \
  --alsologtostderr

✅ After training, the model checkpoints will be available in the `training` folder.
To export the trained model, use the `exporter_main_v2.py` script.